# Example of MeanHamilMinimizer_naive class 

The purpose of this notebook is to describe a naive but pedagogical,
first baby step, in the implementation
of what is called in Qubiter the Mean Hamiltonian Minimization problem.

The qc history of this problem started with quantum chemists planning to 
use on a qc, the phase estimation algo invented by Kitaev? (an algo that is also implemented in Qubiter), to estimate
the energy levels (eigenvalues) of simple molecules, initially H2. Then a
bunch of people realized, heck, rather than trying to estimate the
eigenvalues of a Hamiltonian by estimating the phase changes it causes, we can 
estimate those eigenvalues more efficiently by estimating the mean value of that Hamiltonian
as measured empirically on a qc. One of the first papers to propose this mean idea is https://arxiv.org/abs/1304.3061 Their algo is commonly referred to by the ungainly name VQE (Variational Quantum Eigensolver) VQE was originally applied to do quantum chemistry with a qc. But now Rigetti 
and others have renamed it hybrid quantum-classical quantum computing and pointed out that it's an algo that has wide applicability, not just to quantum chemistry. 

The idea behind hybrid quantum-classical is very simple. One has a classical box CBox and a quantum box QBox. The gates of QBox depend on N gate parameters. QBox sends info to CBox. CBox sends back to QBox N new gate parameters that will lower some cost function. This feedback process between CBox and QBox continues until the cost is minimized. The cost function
is the mean value of a Hamiltonian which is estimated empirically on a qc which resides inside the QBox. 


To minimize a function of N continuous parameters, one can use
some methods like simulated annealing that do not require calculating derivatives,
or one can use methods that do use derivatives.

The family of steepest descent methods (e.g., those
that use auto differentiation and back-propagation) use only first  derivatives.

Other methods use both first (Jacobian) and second (Hessian)  derivatives.
Powell and Nelder-Mead are two methods of this kind. They are both implemented
in the Numpy function `scipy.optimize.minimize`.

Among the methods that do use derivatives,
the performance of those that use both 1st and 2nd  derivatives 
degrades quickly as N grows. Besides, calculating 2nd derivatives
is very expensive. Hence, methods that use the 2nd  derivatives are practically useless in
the neural network field where N is usually very large. In that field, back-propagation rules

Qubiter's class `MeanHamilMinimizer_naive` is a wrapper class designed 
mainly around `scipy.optimize.minimize`. This scipy umbrella method 
implements many of those methods that use 1st and 2nd derivatives. That 
is why I call this class naive. So why write and study this class at 
all? Because it is pedagogical and allows one to set up some of the 
structures used to implement more advanced methods that do use 
back-propagation. 

So, without further ado, here is an example of the use of 
class `MeanHamilMinimizer_naive`

test: $\bra{\psi}M\ket{\phi}$

First change the directory to the Qubiter directory and add it to the path environment variable

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/jupyter/Notebooks/Quantum/qubiter/jupyter-notebooks
/home/jupyter/Notebooks/Quantum/qubiter


Next we construct a simple 4 qubit circuit that depends on two placeholder variables
`#1` and `#2`. These are the continuous variables that we will vary
in  to minimize a cost function. The cost function will be specified later.

In [2]:
from adv_applications.MeanHamilMinimizer_naive import *

In [3]:
num_bits = 4
file_prefix = 'io_folder/mean_hamil_naive_test'
emb = CktEmbedder(num_bits, num_bits)
wr = SEO_writer(file_prefix, emb)
wr.write_Rx(2, rads=np.pi/7)
wr.write_Rx(1, rads='#2*.5')
wr.write_Rn(3, rads_list=['#1', '-#1*3', '#2'])
wr.write_Rx(1, rads='-my_fun#2#1')
wr.write_cnot(2, 3)
wr.close_files()

The code above wrote inside Qubiter's `io_folder`, two files,
an English file and a Picture file. We next ask the writer object 
wr to print those two files for us

In [4]:
wr.print_eng_file()

ROTX	25.714285714285715	AT	2
ROTX	#2*.5	AT	1
ROTN	#1	-#1*3	#2	AT	3
ROTX	-my_fun#2#1	AT	1
SIGX	AT	3	IF	2T



In [5]:
wr.print_pic_file()

|   Rx  |   |   
|   |   Rx  |   
R   |   |   |   
|   |   Rx  |   
X---@   |   |   



The circuit depends on a placeholder function called `my_fun`.
This function will remain fixed during the
minimization process but it must be defined 
and it must be passed in as an input dictionary to 
the constructor of class `MeanHamilMinimizer_naive`

In [6]:
def my_fun(x, y):
    return x + .5*y

fun_name_to_fun = {'my_fun': my_fun}

The initial values for parameters `#1` and `#2` to be minimized
must also be passed in as an input dictionary to 
the constructor of class `MeanHamilMinimizer_naive`

In [7]:
init_var_num_to_rads = {1: np.pi/6, 2: np.pi/3}

One must also pass into the constructor of `MeanHamilMinimizer_naive`, the value of a Hamiltonian called `hamil`.
Qubiter stores `hamil` in an object of the class `QubitOperator`
from the open source Python library `OpenFermion`.
Note that  the `QubitOperator` 
constructor simplifies `hamil` automatically.
Hamiltonians are Hermitian, so after `QubitOperator`
finishes simplifying `hamil`, the coefficient of every "term"
of `hamil` must be real.

In [8]:
hamil = QubitOperator('X1 Y3 X1 Y1', .4) + QubitOperator('Y2 X1', .7)
print('hamil=\n', hamil)

hamil=
 0.7 [X1 Y2] +
0.4 [Y1 Y3]


So what is the purpose of this Hamiltonian `hamil`?
The cost function to be minimized is defined as the mean value of `hamil`.
More precisely, if $\ket{\psi}$ is the ouput of the circuit
specified above, then the cost function equals $\bra{\psi} | H | \ket{\psi}$.

For convenience, we define a function `case()` that creates an
object of `MeanHamilMinimizer_naive`, asks that object ot minimize the cost function,
and then `case()` returns the final result of that minimization process.
As advertised at the beginning of this notebook, we 
use the 'Powell' method as implemented by `scipy.optimize.minimize`

In [9]:
minimizer_fun = scipy.optimize.minimize
min_method = 'Powell'
num_samples = 0
rand_seed = 1234
print_hiatus = 25
verbose = False

def case():
    return MeanHamilMinimizer_naive(file_prefix, num_bits, hamil,
        init_var_num_to_rads, fun_name_to_fun,
        minimizer_fun, num_samples, rand_seed,
        print_hiatus, verbose, method=min_method).find_min()


We focus on two cases, num_samples=0 and num_samples much larger than 0. 
When `MeanHamilMinimizer_naive` is told that 
num_samples=0, it does no sampling. It just calculates
the mean value of `hamil` "exactly', using the exact
final state vector calculated by Qubiter's `SEO_simulator` class

In [10]:
num_samples = 0
case()

iter= 25 , cost= -0.111919833534 , x_val= [ 2.70495801  1.89906689]
iter= 50 , cost= -0.156937579192 , x_val= [ 2.54047272  2.19314244]
iter= 75 , cost= -0.163946013152 , x_val= [ 2.4709106   2.50397983]
iter= 100 , cost= -0.163696596963 , x_val= [ 2.4568085  2.5726661]


   direc: array([[-0.11902267, -0.046481  ],
       [-0.02985049,  0.16432022]])
     fun: -0.16395379241629054
 message: 'Optimization terminated successfully.'
    nfev: 104
     nit: 4
  status: 0
 success: True
       x: array([ 2.4680443 ,  2.51081554])

When num_samples > 0, Qubiter calculates the final state vector
of the circuit, then it samples that num_samples times,
obtains an empirical probability distribution from that,
and then it calculates the mean value of `hamil` 
using that empirical distribution

In [11]:
num_samples = 10000
case()

iter= 25 , cost= -0.12618 , x_val= [ 2.68613585  2.00152068]
iter= 50 , cost= -0.16292 , x_val= [ 2.56254065  2.2849027 ]
iter= 75 , cost= -0.16938 , x_val= [ 2.51384103  2.4406475 ]
iter= 100 , cost= -0.17306 , x_val= [ 2.48602502  2.48474038]
iter= 125 , cost= -0.174 , x_val= [ 2.48133282  2.53781611]
iter= 150 , cost= -0.17138 , x_val= [ 2.49542984  2.51135333]
iter= 175 , cost= -0.1732 , x_val= [ 2.48273557  2.52681542]
iter= 200 , cost= -0.17442 , x_val= [ 2.48524374  2.50685823]


   direc: array([[-0.12359519, -0.0545423 ],
       [-0.01719124,  0.13678867]])
     fun: -0.17442000000000002
 message: 'Optimization terminated successfully.'
    nfev: 212
     nit: 5
  status: 0
 success: True
       x: array([ 2.48524374,  2.50685822])